## Measuring VNC length

The VNC length is measured using a centerline estimation.
The steps are as follows:

1. The image is binarized, so we can apply a distance transform.
The distance transform is very succeptible to holes and concavities, so we also remove holes from the binary image and apply a binary opening to smooth edges.
2. We apply a 'chessboard' distance transform (minimum difference between two points along any coordinate dimension) to the binary image, which naturally results in peaks over the centerline of the binary image.
Given the embryo morphology, we expect several center points along the anterior-posterior axis and eventually another (smaller) set of points towards the brain lobes.
3. To separate these points, we fit a RANSAC model that, given the majority of the points are along the AP axis, fits a line along the VNC centerline.
4. From this line intersection to the binary image, we are able to measure the VNC length.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
from tifffile import imread

from pasnascope import centerline, find_hatching, utils, vnc_length

experiment_name = '20240515'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)
print(f"Selected data for experiment: {experiment_name}")

Visualize centerline method for a single frame. Includes ROI, inliers (green), outliers(red), and centerline.
Used to better understand the method and how the VNC length is actually calculated.

Change the `frame_number` variable to change the frame that is displayed for a given embryo.
Change `i` to select a different embryo.

In [ ]:
i = 0
frame_number = 100

img_dir = project_dir.joinpath('embs')
imgs = sorted(img_dir.glob('*ch2.tif'), key=utils.emb_number)

img_path = imgs[i]
img = imread(img_path, key=frame_number)
img_title = f"{img_path.stem} - Frame {frame_number}"
centerline.view_centerline_dist(img, img_title, thres_rel=0.6, min_dist=5)

Plot VNC length for a single embryo.

Change `i` to select a different embryo.

In [ ]:
i = 0
interval = 20

img_dir = project_dir.joinpath('embs')
embs = sorted(img_dir.glob('*-ch2.tif'), key=utils.emb_number)
emb = embs[i]

hp = find_hatching.find_hatching_point(emb)
hp -= hp % interval

img = imread(emb, key=range(0,hp, interval))
vnc_len = vnc_length.measure_VNC_centerline(img)
x = list(range(0, len(vnc_len)*interval, interval))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle(f'VNC length estimation - {emb.stem}')

ax.plot(x, vnc_len);

Calculate the VNC length for all embryos of the current experiment, and save as a csv file in the `results` directory.

In [ ]:
file_name = 'lengths.csv'

interval = 20
img_dir = project_dir.joinpath('embs')
embs = sorted(img_dir.glob('*ch2.tif'), key=utils.emb_number)

dest = root_dir.joinpath('results', experiment_name)
dest.mkdir(parents=True, exist_ok=True)
file_path = dest.joinpath(file_name)

if file_path.exists():
    print(f"The file `{file_name}` already exists, and won't be overwritten.\nChoose another file or delete the existing file.")
else:
    lengths = []
    ids = []

    for emb in embs:
        print(f"Processing {emb.stem}..")
        hp = find_hatching.find_hatching_point(emb)
        hp -= hp % interval

        img = imread(emb, key=range(0,hp, interval))
        vnc_len = vnc_length.measure_VNC_centerline(img)
        lengths.append(vnc_len)
        ids.append(utils.emb_number(emb))

    saved = vnc_length.export_csv(ids, lengths, file_path, interval)
    if saved:
        print(f"Wrote length data for {len(embs)} embryos, at {file_path}")